# This Notebook will be used for Toronto Assignment of Capstone Project on Coursera

## 1. Creating the Pandas Data Frame with the Required Data

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
#scapping wiki pange to get the required table
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page=requests.get(url).text

soup= BeautifulSoup(wiki_page,'html.parser')
req_table=soup.find('table',class_="wikitable sortable")
# req_table.findAll('th')

In [3]:
#parsing the html to store table columns in a list
PC=[]
Borough=[]
Neighborhood=[]

for data in req_table.findAll('tr'):
    values= data.findAll('td')
    if len(values):
        PC.append(values[0].find(text=True).strip())
        Borough.append(values[1].find(text=True).strip())
        Neighborhood.append(values[2].find(text=True).strip())
        

In [4]:
#saving the table data in dataframe from list
postal_df=pd.DataFrame()
postal_df['PostalCode']=PC
postal_df['Borough']=Borough
postal_df['Neighborhood']=Neighborhood
postal_df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [5]:
# removing rows containing 'Non Assigned' data
postal_df=postal_df[postal_df['Borough']!='Not assigned'].reset_index(drop='True')

In [6]:
# checking if any Neighborhood are still 'Not Asigned'
postal_df[postal_df['Neighborhood']=='Not assigned'].reset_index(drop='True')

,PostalCode,Borough,Neighborhood


### since no 'Non Assigned' rows were found ,we are not replacing Neighborhood with Borough name for those rows

In [7]:
#Print the shape of df
postal_df.shape

(103, 3)

In [8]:
#print df head
postal_df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## 2. Adding Lat Lon columns to the Pandas Data Frame

In [9]:
import geocoder # import geocoder

In [10]:
# inserting the given code in a function (limit of 50 requests per postal code was done to check the viability of the soln)
def getLatLon(row):
#     print(row)
    postal_code=row.PostalCode
    print(postal_code)
    # initialize your variable to None
    lat_lng_coords = None
    tries=0
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
      tries+=1
      if tries % 5 ==0:
            print("Tried "+str(tries)+" times")
            return "Not found -- Exiting"
    print(lat_lng_coords,tries)
    
    return lat_lng_coords
#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]

In [11]:
# calling the function to check how long it takes to get lat lon for a postal code
postal_df.apply(getLatLon, axis=1)

M3A
Tried 5 times
M4A
Tried 5 times
M5A
Tried 5 times
M6A
Tried 5 times
M7A
Tried 5 times
M9A
Tried 5 times
M1B
Tried 5 times
M3B
Tried 5 times
M4B
Tried 5 times
M5B
Tried 5 times
M6B
Tried 5 times
M9B
Tried 5 times
M1C
Tried 5 times
M3C
Tried 5 times
M4C
Tried 5 times
M5C
Tried 5 times
M6C
Tried 5 times
M9C
Tried 5 times
M1E
Tried 5 times
M4E
Tried 5 times
M5E
Tried 5 times
M6E
Tried 5 times
M1G
Tried 5 times
M4G
Tried 5 times
M5G
Tried 5 times
M6G
Tried 5 times
M1H
Tried 5 times
M2H
Tried 5 times
M3H
Tried 5 times
M4H
Tried 5 times
M5H
Tried 5 times
M6H
Tried 5 times
M1J
Tried 5 times
M2J
Tried 5 times
M3J
Tried 5 times
M4J
Tried 5 times
M5J
Tried 5 times
M6J
Tried 5 times
M1K
Tried 5 times
M2K
Tried 5 times
M3K
Tried 5 times
M4K
Tried 5 times
M5K
Tried 5 times
M6K
Tried 5 times
M1L
Tried 5 times
M2L
Tried 5 times
M3L
Tried 5 times
M4L
Tried 5 times
M5L
Tried 5 times
M6L
Tried 5 times
M9L
Tried 5 times
M1M
Tried 5 times
M2M
Tried 5 times
M3M
Tried 5 times
M4M
Tried 5 times
M5M
Tried 

0      Not found -- Exiting
1      Not found -- Exiting
2      Not found -- Exiting
3      Not found -- Exiting
4      Not found -- Exiting
               ...         
98     Not found -- Exiting
99     Not found -- Exiting
100    Not found -- Exiting
101    Not found -- Exiting
102    Not found -- Exiting
Length: 103, dtype: object

In [15]:
### Since it takes too long to get values for any postal code , we dwnload the given csv

In [16]:
# reading the downloaded csv
latLon_df=pd.read_csv(r'C:\Users\eutsaga\Downloads\Geospatial_Coordinates.csv')
latLon_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#merging the new df with the existing df to create a new df
postal_df2=pd.merge(postal_df,latLon_df, left_on='PostalCode',right_on='Postal Code').drop('Postal Code',axis=1)
postal_df2.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## 3. Exploring Toronto neighborhoods

In [23]:
# function to explore the neighbourhoods 
CLIENT_ID = 'P3Q0EEQIF5A3033ZLP1RLQDIWIUCANZYF3KQZ4LAZC4AAZFA' # Foursquare ID
CLIENT_SECRET = '32RTBTOZRF1DMCPHC1BWVO23B2Q4D4DCX2U2TWHWIQATG1JZ' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print(name)
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:

postal_venues_df = getNearbyVenues(names=postal_df2['Neighborhood'],
                                   latitudes=postal_df2['Latitude'],
                                   longitudes=postal_df2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [25]:
# one hot encoding
postal_onehot = pd.get_dummies(postal_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
postal_onehot['Neighborhood'] = postal_venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [postal_onehot.columns[-1]] + list(postal_onehot.columns[:-1])
postal_onehot = postal_onehot[fixed_columns]

postal_onehot.head()

NameError: name 'postal_venues_df' is not defined

In [ ]:
postal_onehot = postal_onehot.groupby('Neighborhood').mean().reset_index()
postal_onehot.head()

### Getting top venues per neighborhood and converting it into 1 hot 

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = postal_grouped['Neighborhood']

for ind in np.arange(postal_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(postal_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Running KMeans Clustering

In [138]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

postal_grouped_clustering = postal_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(postal_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [141]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

postal_merged = postal_df2

# merge postal_grouped with postal_data to add latitude/longitude for each neighborhood
postal_merged = postal_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

postal_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Bus Stop,Park,Food & Drink Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dim Sum Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Hockey Arena,Portuguese Restaurant,Pizza Place,Coffee Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Chocolate Shop,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Boutique,Miscellaneous Shop,Coffee Shop,Event Space,Vietnamese Restaurant,Accessories Store,Ethiopian Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,College Cafeteria,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café


### Potting the clusters

In [163]:
import folium # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude=43.6532
longitude=-79.3832
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

postal_merged.dropna(inplace=True)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(postal_merged['Latitude'], postal_merged['Longitude'], postal_merged['Neighborhood'], postal_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters